In [ ]:
import pandas as pd
import os
directory_path = 'data\QnAs'

file_list = [os.path.join(directory_path, filename) for filename in os.listdir(directory_path) if filename.endswith('.txt')]
print(len(file_list))

In [ ]:
def file_to_df(filenames):

    # Initialize empty lists for questions and answers
    questions = []
    answers = []

    # Loop through each file and extract questions and answers
    for filename in filenames:
        with open(filename, 'r', encoding='utf-8') as f:
            data = [line.strip() for line in f]

        # Loop through each line and append to respective lists
        for i, line in enumerate(data):
            if line.startswith('Q:'):
                questions.append(line[2:])
            elif line.startswith('A:'):
                # Keep appending lines until the next question is encountered
                answer = line[2:]
                j = i + 1
                while j < len(data) and not data[j].startswith('Q:'):
                    answer += ' ' + data[j]
                    j += 1
                answers.append(answer)

    # Create pandas dataframe
    print(len(questions))
    print(len(answers))
    df = pd.DataFrame({'question': questions, 'answer': answers})
    # Add 'id' column as the first column
    # df.insert(0, 'id', range(1, len(df) + 1))

    # Print dataframe
    print(f"Objects in DataFrame: {len(df.question)}")
    return df


In [ ]:
df.to_csv('data.csv')

In [ ]:
file_list[40]

In [ ]:
df = file_to_df(file_list)

In [ ]:
import weaviate
import cohere
import json
auth_config = weaviate.auth.AuthApiKey(api_key="<<Your API Key>>")
client = weaviate.Client(
    url = "Replace with your endpoint", 
    auth_client_secret=auth_config,
    additional_headers = {
        "X-Open-Api-Key" : "<<Your API Key>>" # Replace with your API key
    }  
)

In [ ]:

client.schema.get()
json_data = df.to_json(orient='records')
class_obj = {
    "class": "Question",
    "vectorizer": "text2vec-huggingface"  # Or "text2vec-cohere" or "text2vec-huggingface"
}

client.schema.create_class(class_obj)
data = json.loads(json_data)
with client.batch as batch:
    batch.batch_size=100
    for i, d in enumerate(data):
        properties = {
            "answer": d["answer"],
            "question": d["question"]
        }

        client.batch.add_data_object(properties, "Question")

In [ ]:
client.schema.get()